In [1]:
# Artefact Data Engineering Test 
# case study 1 - web scraping
# to crawl comments from one jd.com product page

In [9]:
# load packages
import requests
import urllib3
import json
import urllib
import urllib.request
from bs4 import BeautifulSoup

In [6]:
# the provided source link : https://item.jd.com/1384071.html
# there are 100 pages comments in total
# at first we try to tune the code by scraping 10 pages of comments then save json files to local
for i in range(1, 11):
    url1 = 'https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98vv376&productId=1384071&score=0&sortType=5&page='
    url2 = str(i)
    uel3 = '&pageSize=10&isShadowSku=0&rid=0&fold=1'
    finalurl = url1+url2+uel3
    xba = requests.get(finalurl)

for i in range(1, 11):
    # u1 = "~/Desktop/crawl/"
    u2 = str(i)
    u3 = ".json"
    finalu = u2+u3
    file = open(finalu, "w")
    file.write(xba.text[26:-2])
    file.close()
st = xba.text
print('finished')

finished


In [7]:
import pandas as pd
import urllib.request as req
import json
import sys
import time
import random

print(sys.getdefaultencoding())

class JDCommentsCrawler:
    
    def __init__(self,productId=None,callback=None,page=1,score=0,sortType=5,pageSize=10):
        self.productId = productId # product ID
        self.score = score # score type（good：3, average：2, fair：1, all：0）
        self.sortType = sortType # sort type（recommend：5, time：6）
        self.pageSize = pageSize # comments in a page（default:10）
        self.callback = callback # call back parameter, checked by every product
        self.page = page
        self.locationLink = 'https://sclub.jd.com/comment/productPageComments.action'
        self.paramValue = {
            'callback':self.callback,
            'productId':self.productId,
            'score':self.score,
            'sortType':self.sortType,
            'pageSize':self.pageSize,
        }        
        self.locationUrl = None
    def paramDict2Str(self,params):        
        str1 = ''
        for p,v in params.items():
            str1 = str1+ p+'='+str(v)+'&'
        return str1
    def concatLinkParam(self):
        self.locationUrl = self.locationLink+'?'+self.paramDict2Str(self.paramValue)+'isShadowSku=0&fold=1&page=0'
        #print(self.locationUrl)
        
    def requestMethod(self):
        headers = {
            'Connection': 'Keep-Alive',
            'Accept': 'text/html, application/xhtml+xml, */*',            
            'Accept-Language': 'zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:60.0) Gecko/20100101 Firefox/60.0',
            'Referer':'https://item.jd.com/%d.html'%(self.productId),
            'Host':'sclub.jd.com'          
        }
        reqs = req.Request(self.locationUrl,headers=headers)
        print('reqs : ',reqs)
        return reqs       
    def showList(self):
        request_m = self.requestMethod()       
        conn = req.urlopen(request_m)
        return_str = conn.read().decode('gbk')
        return_str = return_str[len(self.callback)+1:-2]
        return json.loads(return_str)   
    def requestMethodPage(self,p):
        # set the headers and open links
        headers = {
            'Connection': 'Keep-Alive',
            'Accept': 'text/html, application/xhtml+xml, */*',            
            'Accept-Language': 'zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:60.0) Gecko/20100101 Firefox/60.0',
            'Referer':'https://item.jd.com/%d.html'%(self.productId),
            'Host':'sclub.jd.com'          
        }
        url = self.locationUrl[:-1]+str(p)
        print('url : ',url)
        reqs = req.Request(url,headers=headers)
        return reqs
    def showListPage(self,p):
        request_m = self.requestMethodPage(p)       
        conn = req.urlopen(request_m)
        return_str = conn.read().decode('gbk')
        return_str = return_str[len(self.callback)+1:-2]
        return json.loads(return_str)
    def save_csv(self,df,p):
        # save documents as 'jd_i.csv'
        df.to_csv(path_or_buf = '/Users/mac/Desktop/crawl/jd_%d.csv'%p,encoding='gbk') 
 
    def crawler(self):
        # save comments data to CSV files
        dfs = []
        for p in range(self.page):
            json_info = self.showListPage(p)
            tmp_list = []
            #print(json_info)
            productCommentSummary = json_info['productCommentSummary']
            productId = productCommentSummary['productId']
            comments = json_info['comments']
            for com in comments:
                tmp_list.append([com['id'],productId,com['guid'],com['content'],com['creationTime'],com['referenceId'],com['referenceTime'],com['score'],\
                                 com['nickname'],com['userLevelName'],com['isMobile'],com['userClientShow']])
            df = pd.DataFrame(tmp_list,columns=['comment_id','product_id','guid','content','create_time','reference_id','reference_time','score',\
                                                'nickname','user_level','is_mobile','user_client'])
            self.save_csv(df,p)
            dfs.append(df)
            time.sleep(random.randint(31,52))
        final_df = pd.concat(dfs,ignore_index=True)
        self.save_csv(final_df,self.page)
 
def jdComment():
    # set parameters according to the target website
    page =  100  # page numbers
    productId = 1384071 # product ID
    callback = 'fetchJSON_comment98vv376' # call back parameters
    JDC = JDCommentsCrawler(productId,callback,page)
    JDC.concatLinkParam()
    JDC.crawler()

if __name__ == '__main__':
    jdComment()

print('finished')

utf-8
url :  https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98vv376&productId=1384071&score=0&sortType=5&pageSize=10&isShadowSku=0&fold=1&page=0
url :  https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98vv376&productId=1384071&score=0&sortType=5&pageSize=10&isShadowSku=0&fold=1&page=1
url :  https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98vv376&productId=1384071&score=0&sortType=5&pageSize=10&isShadowSku=0&fold=1&page=2
url :  https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98vv376&productId=1384071&score=0&sortType=5&pageSize=10&isShadowSku=0&fold=1&page=3
url :  https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98vv376&productId=1384071&score=0&sortType=5&pageSize=10&isShadowSku=0&fold=1&page=4
url :  https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98vv376&productId=1384071&score=0&s

url :  https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98vv376&productId=1384071&score=0&sortType=5&pageSize=10&isShadowSku=0&fold=1&page=47
url :  https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98vv376&productId=1384071&score=0&sortType=5&pageSize=10&isShadowSku=0&fold=1&page=48
url :  https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98vv376&productId=1384071&score=0&sortType=5&pageSize=10&isShadowSku=0&fold=1&page=49
url :  https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98vv376&productId=1384071&score=0&sortType=5&pageSize=10&isShadowSku=0&fold=1&page=50
url :  https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98vv376&productId=1384071&score=0&sortType=5&pageSize=10&isShadowSku=0&fold=1&page=51
url :  https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98vv376&productId=1384071&score=0&so

url :  https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98vv376&productId=1384071&score=0&sortType=5&pageSize=10&isShadowSku=0&fold=1&page=94
url :  https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98vv376&productId=1384071&score=0&sortType=5&pageSize=10&isShadowSku=0&fold=1&page=95
url :  https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98vv376&productId=1384071&score=0&sortType=5&pageSize=10&isShadowSku=0&fold=1&page=96
url :  https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98vv376&productId=1384071&score=0&sortType=5&pageSize=10&isShadowSku=0&fold=1&page=97
url :  https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98vv376&productId=1384071&score=0&sortType=5&pageSize=10&isShadowSku=0&fold=1&page=98
url :  https://sclub.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98vv376&productId=1384071&score=0&so

In [ ]:
# mysql database
import pymysql

db_connection = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='xxxx',
    database='jdcomm',
    charset='utf8'
)